# SimFin - Small Recipes Cookbook
This is a collection of small recipes for users that are already familiar with the [basics](https://github.com/simfin/simfin-tutorials/blob/master/01_Basics.ipynb) (will be extended steadily with new examples).

## Imports

In [1]:
# Import the main functionality from the SimFin Python API.
import simfin as sf

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

## Config

In [2]:
# SimFin data-directory.
sf.set_data_dir('~/simfin_data/')

# SimFin load API key or use free data.
sf.load_api_key(path='~/simfin_api_key.txt', default_key='free')

## Load Datasets

In these examples, we will use the following datasets:

In [5]:
# Data for USA.
market = 'us'

# ticker: MSFT (Microsoft)
ticker = 'MSFT'

# Load TTM Income Statements.
df_inc = sf.load_income(variant='ttm', market=market).loc[ticker]

# Load TTM Cash-Flow Statements.
df_cf = sf.load_cashflow(variant='ttm', market=market).loc[ticker]

Dataset "us-income-ttm" on disk (1 days old), loading.
Dataset "us-cashflow-ttm" on disk (1 days old), loading.


## Recipes

### Calculate EBITDA

In [7]:
# Calculates EBITDA for ticker
df_ebitda = df_inc[OP_INCOME_LOSS] + df_cf[DEPR_AMOR]

print(df_ebitda)

Report Date
2010-03-31    2.450100e+10
2010-06-30    2.677100e+10
2010-09-30    2.945300e+10
2010-12-31    2.915300e+10
2011-03-31    2.971500e+10
2011-06-30    2.992700e+10
2011-09-30    3.004600e+10
2011-12-31    2.989000e+10
2012-03-31    3.060100e+10
2012-06-30    3.092300e+10
2012-09-30    2.901200e+10
2012-12-31    2.912000e+10
2013-03-31    3.064500e+10
2013-06-30    3.051900e+10
2013-09-30    3.178900e+10
2013-12-31    3.223900e+10
2014-03-31    3.180300e+10
2014-06-30    3.309800e+10
2014-09-30    3.422200e+10
2014-12-31    3.453200e+10
2015-03-31    3.460200e+10
2015-06-30    3.412900e+10
2015-09-30    3.297100e+10
2015-12-31    3.100100e+10
2016-03-31    2.969200e+10
2016-06-30    3.381000e+10
2016-09-30    3.508700e+10
2016-12-31    3.758800e+10
2017-03-31    3.977400e+10
2017-06-30    3.810900e+10
2017-09-30    3.978500e+10
2017-12-31    4.092900e+10
2018-03-31    4.275500e+10
2018-06-30    4.531900e+10
2018-09-30    4.790400e+10
2018-12-31    4.994200e+10
2019-03-31    5.